In [1]:
# Initial imports and enviroment setup
import gymnasium as gym
import numpy as np
import sys
import json
from collections import defaultdict
import matplotlib.pyplot as plt

# create Blackjack environment
env = gym.make("Blackjack-v1", natural=False, sab=False)

Check [this](https://gymnasium.farama.org/environments/toy_text/blackjack/) for more information about the Blackjack environment. 

In addition, check [this](https://youtu.be/NeusGkowXR4?si=9a1aE_bInK4vSAHw) and [this](https://youtu.be/wn8hlPNwL74?si=PV_h3WQCXmZKwRzW) videos for how the below code is written.

### Action Space

- 0: Stick
- 1: Hit

### Starting State

| **Observation** | **Values** |
| --- | --- |
| Player current sum | $4, 5, \cdots, 21$ |
| Dealer showing card value | $1, 2, \cdots, 11$ |
| Usable Ace | $0, 1$ |

### Rewards

- win game: +1
- lose game: -1
- draw game: 0
- win game with natural blackjack: +1.5 (if `natural` is True) +1 (if `natural` is False)

### Episode End

The episode ends if the following happens:

- Termination:

1. The player hits and the sum of hand exceeds 21.
2. The player sticks.

An ace will always be counted as usable (11) unless it busts the player.

Our Q function will map 3 states to 2 actions, ie, $\text{Q}: \; 3 \text{ states} \; \rightarrow \; 2 \text{ actions}$

Check [this](https://www.geeksforgeeks.org/defaultdict-in-python/) for a refresher on `defaultdict`.

In [2]:
env.action_space

Discrete(2)

In [3]:
# number of different discrete actions
env.action_space.n

2

In [4]:
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [8]:
# env.reset()
env.reset()[0]

(13, 10, 0)

In [5]:
Q = defaultdict(lambda: np.zeros(env.action_space.n))

The Q function takes as input the 3 different state values:
- player's current
- dealer's first card
- whether the player has a usable ace

Based on these 3 state values, the Q function returns 2 action values:
- value of action 0, ie, the action-value of playing 'stick'.
- value of action 1, ie, the action-value of playing 'hit'. 

In [6]:
Q[env.reset()[0]]

array([0., 0.])

In [ ]:
def epsilon_greedy_policy(Q, state, epsilon):
    probs = np.zeros(env.action_space.n)
    optimal_action = np.argmax(Q[state])
    sub_optimal_action = 
    pass

In [ ]:
def generate_episode():
    pass

In [10]:
class Kamal_MC:
    def __init__(self, num_episodes, epsilon, epsilon_min, alpha, decay_rate):
        """
        param num_episodes: (int) number of episodes to run the algorithm
        param epsilon: (float) 
        param epsilon_min: (float) 
        param alpha: (float)
        param decay_rate: (float)
        """
        self.num_episodes = num_episodes
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.alpha = alpha
        self.decay_rate = decay_rate
        self.Q = defaultdict(lambda: np.zeros(env.action_space.n))

    def monte_carlo_control(self):
        for episode in range(1, self.num_episodes + 1):
            self.epsilon = max(self.epsilon_min, self.epsilon * self.decay_rate)
            

In [ ]:
# hyperparameters
num_episodes = 10_000
epsilon = 1
epsilon_min = 0.05
alpha = 0.02
decay_rate = 0.9999

mcc = Kamal_MC(num_episodes, epsilon, epsilon_min, alpha, decay_rate)

In [8]:
class GLIE_MC_Control:
    def __init__(self):
        pass

    def every_visit(self):
        """
        This is the every-visit GLIE Monte Carlo Control algorithm.
        Check pg. 88 of Nimish Sanghi's book for the pseudocode.
        """
        pass

    def first_visit(self):
        """
        This is the first-visit GLIE Monte Carlo Control algorithm.
        """
        pass

In [4]:
def fn():
    state, _ = env.reset()
    print("Initial state:\t", state)

fn()

Initial state:	 (15, 10, 0)
